In [15]:
import findspark

In [16]:
findspark.init('/home/kant/spark-2.4.4-bin-hadoop2.7')

In [17]:
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession.builder.appName('Cruise').getOrCreate()

In [19]:
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [20]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [21]:
for row in data.head(5):
    print(row)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)




In [22]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [24]:
from pyspark.ml.feature import StringIndexer

In [25]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_cat')
indexed = indexer.fit(data).transform(data)

In [26]:
indexed.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_cat=16.0)]

In [28]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [30]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_cat']

In [32]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'Cruise_cat'], outputCol='features')

In [33]:
output = assembler.transform(indexed)

In [34]:
output.select('features','crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [35]:
final_data = output.select(['features','crew'])

In [37]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [38]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               108|
|   mean| 7.764351851851861|
| stddev|3.6675254009525347|
|    min|               0.6|
|    max|              21.0|
+-------+------------------+



In [39]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               50|
|   mean|7.858600000000001|
| stddev|3.154364006533534|
|    min|             0.59|
|    max|             13.6|
+-------+-----------------+



In [40]:
from pyspark.ml.regression import LinearRegression

In [42]:
cruise_lr = LinearRegression(labelCol='crew')

In [43]:
trained_ship_model = cruise_lr.fit(train_data)

In [54]:
predicted = trained_ship_model.transform(test_data)

In [55]:
predicted.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[5.0,160.0,36.34,...| 13.6|15.078691571592818|
|[6.0,90.0,20.0,9....|  9.0| 10.18458863764295|
|[6.0,110.23899999...| 11.5|11.324201016360607|
|[6.0,112.0,38.0,9...| 10.9| 11.45319133039536|
|[7.0,89.6,25.5,9....| 9.87|11.321921308036366|
|[8.0,110.0,29.74,...| 11.6| 12.34463118921849|
|[9.0,105.0,27.2,8...|10.68|11.387002960634526|
|[9.0,113.0,26.74,...|12.38|  11.4026769587892|
|[10.0,77.0,20.16,...|  9.0| 8.907547031371301|
|[10.0,91.62700000...|  9.0|  9.23096253098781|
|[10.0,151.4,26.2,...|12.53|10.785768128217414|
|[11.0,58.6,15.66,...|  7.6|  7.56855471176981|
|[11.0,86.0,21.24,...|  9.3|  9.65394720991953|
|[12.0,58.6,15.66,...|  7.0| 7.550853450578662|
|[12.0,88.5,21.24,...|10.29| 9.572525776029057|
|[12.0,88.5,21.24,...|  9.3|10.556889411446459|
|[12.0,91.0,20.32,...| 9.99| 9.154480018981976|
|[12.0,138.0,31.14...|11.85|12.914642763

In [44]:
trained_result = trained_ship_model.evaluate(test_data)

In [46]:
trained_result.rootMeanSquaredError

0.8771026390054414

In [48]:
trained_result.r2

0.9211046980360093

In [49]:
trained_result.meanAbsoluteError

0.6659061251892546

In [52]:
from pyspark.sql.functions import corr